In [3]:
import random
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as Acivation
from sklearn.model_selection import train_test_split

from torch import nn
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/work1/tdata.zip', 'r')
file = zip_ref.extractall("./tmp")
zip_ref.close()

print("o")

o


In [13]:
class CNN(nn.Module):

    def __init__(self,vgg_features):
        super().__init__()

        self.features = vgg_features

    def forward(self, z):
        z = self.features(z)
        z = z.view(-1, 512*7*7)

        return z


class CNN_LSTM(nn.Module):

    def __init__(self, vgg_features):
        super(CNN_LSTM, self).__init__()
        self.cnn = CNN(vgg_features)
        self.rnn = nn.LSTM(
            input_size=512*7*7, 
            hidden_size=64, 
            num_layers=1,
            batch_first=True,
            bidirectional=True)
        self.linear = nn.Linear(128,10)

    def forward(self, x):
        batch_size, timesteps, C, H, W = x.size()
        c_in = x.view(batch_size * timesteps, C, H, W)
        c_out = self.cnn(c_in)
        r_in = c_out.view(batch_size, timesteps, -1)
        r_out, (h_n, h_c) = self.rnn(r_in)
        r_out2 = self.linear(r_out[:, -1, :])
        
        return F.log_softmax(r_out2, dim=1)

In [14]:
vgg_f =  models.vgg16(pretrained=True).features
model = CNN_LSTM(vgg_f)


In [15]:
# load model
PATH = '/content/drive/MyDrive/work2/bimodel_weights_lstm.pth'
model = torch.load(PATH)
# print(model)

In [122]:
from PIL import Image


transform1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5,))

])

CATEGORIES=["Tomato___Bacterial_spot","Tomato___Early_blight",
            "Tomato___healthy","Tomato___Late_blight",
            "Tomato___Leaf_Mold","Tomato___Septoria_leaf_spot",
            "Tomato___Spider_mites Two-spotted_spider_mite","Tomato___Target_Spot",
            "Tomato___Tomato_mosaic_virus","Tomato___Tomato_Yellow_Leaf_Curl_Virus"]
        
path = '/content/drive/MyDrive/work2/imgs/septoria/ (5).JPG'
img = Image.open(path)
# img.shape
img = transform1(img)
img.shape

image = img.view(1,1,img.shape[0], img.shape[1], img.shape[2])
image.shape


output = model(image.to(device)).to(device)
_,pred=torch.max(output,1)
print(pred[0].item())
print(CATEGORIES[pred[0].item()])

0
Tomato___Bacterial_spot
